In [82]:
## Gerekli Kütüphaneler
import pandas as pd
import numpy as np
import warnings
## Excel düzenlediğimiz için hata koduna ihtiyacımız yok
warnings.filterwarnings('ignore')


In [83]:
key = pd.read_excel('./İl_ilçe_adı_dict_2.xlsx')
key = key.drop_duplicates().reset_index(drop=True)

In [84]:
# Nüfus verisi düzenleme
a = pd.read_csv('./İlçe_Nüfusları_ham.csv', sep='|')
## Gereksiz kısımları sil
a = a.iloc[5:,:3]
## Sütun isimlerini düzelt
a.columns = ['Key',2017,2022]
## Replace NaN with 0
a[2017] = a[2017].replace(np.nan,0)
a[2022] = a[2022].replace(np.nan,0)
## Dropna NA
a = a.dropna().reset_index(drop=True)
## İl-İlçe İsimleri
a['İl'] = [zz.split('(')[0] for zz in a['Key']]
a['İlçe'] = [zz.split('(')[1].split(')')[0] for zz in a['Key']]
## Gerekli Sütunlar
a = a[['İl','İlçe',2017,2022]]
## Verileri sayıya çevirme
a[2017] = [float(zz) for zz in a[2017]]
a[2022] = [float(zz) for zz in a[2022]]
## Merge with key
a = a.merge(key, on =['İl','İlçe'] ,how='left')
## Columns
a = a[['ILADI','ILCEADI',2017,2022]]
## Last Check
a = a.drop_duplicates().reset_index(drop=True)

In [85]:
## SEGE Verileri
## 2017
b = pd.read_excel('./SEGE_2017_ham.xlsx')
b.columns = ['İl','İlçe','Skor 2017']
## Merge with key
b = b.merge(key, on =['İl','İlçe'] ,how='left')
## Columns
b = b[['ILADI','ILCEADI','Skor 2017']]

## Adjust badly read scores
val = list()
for v in b['Skor 2017']:
    try:
        if v >= 1000 or v <= -1000:
            val.append(v/1000)
        else:
            val.append(v)
    except:
        val.append(np.nan)
b['Skor 2017'] = val

## 2022
c = pd.read_excel('./SEGE_2022_ham.xlsx')
c.columns = ['İl','İlçe','Skor 2022']
## Merge with key
c = c.merge(key, on =['İl','İlçe'] ,how='left')
## Columns
c = c[['ILADI','ILCEADI','Skor 2022']]

## Adjust badly read scores
val = list()
for v in c['Skor 2022']:
    try:
        if v >= 1000 or v <= -1000:
            val.append(v/1000)
        else:
            val.append(v)
    except:
        val.append(np.nan)
c['Skor 2022'] = val

b = b.merge(c, how='outer')

In [86]:
## Birleştirilmiş Sonuç
df = a.merge(b, on = ['ILADI','ILCEADI'], how='outer')
## Kaydet
df.to_excel('Eşleştirilmiş.xlsx', index=False)
